# 7장 앙상블 학습과 랜덤 포레스트

- 앙상블: 일련의 예측기(분류 또는 회귀)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측

  (ex) 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정트리 분류기를 훈련시킨후 가장 많은 선택을 받은 클래스를 예측으로 삼음

  - 랜덤 포레스트: 결정 트리의 앙상블
  - 배깅, 부스팅, 스태킹 등

## 7.1 투표 기반 분류기

- 직접 투표 분류기: 다수결 투표로 정해지는 분류기
- 분류기가 약한 학습기라도 충분히 많고 다양하다면, 앙상블은 강한 학습기가 될 수 있음
- 큰 수의 법칙


- 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야 함
- 같은 데이터로 훈련시키면 안됨-분류기들이 같은 종류의 오차를 만들기 쉽기 때문에 잘못된 클래스가 다수인 경우가 많고, 앙상블의 정확도가 낮아짐
- 다양한 분류기를 얻는다=각기 다른 알고리즘으로 학습


- 간접투표: 모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_probs()가 있으면), 개별 분류기의 예측을 평균내어 확률이 가장 높은 클래스를 예측할 수 있음
    - voting="soft"
    - SVC는 기본값에서는 클래스 확률을 제공하지 않으므로, probability 매개변수를 True로 지정해야 함

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.3, random_state=45)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [11]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [12]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.856
RandomForestClassifier 0.904
SVC 0.928
VotingClassifier 0.928


## 7.2 배깅과 페이스팅

- 다양한 분류기 만들기
  - 각기 다른 훈련 알고리즘 사용
  - 같은 알고리즘 사용, 훈련 세트의 서브셋을 무작위 구성하여 분류기 각기 다르게 학습
- 배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅: 중복을 허용하지 않고 샘플링하는 방식


- 배깅과 페이스팅에서느 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있음
- 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링


- 수집함수는 분류일 때는 통계적 최빈값, 회귀일 때는 평균 계산
- 수집함수를 통과하면 편향과 분산이 모두 감소
- 일반적으로 앙싱블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듦



- 예측기는 병렬로 학습시킬 수 있기 때문에,배깅과 페이스팅의 인기 높

### 7.2.1 사이킷런의 배깅과 페이스팅
- 부트 스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높음
- 다양성을 추가한다는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킴
- 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호
- 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가해서 더 나은 쪽을 선택하는 것이 좋음

In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=100,bootstrap=True,n_jobs=-1)
#n_jobs=-1: 가용한 모든 코어 사용
bag_clf.fit(X_train,y_train)
y_pred=bag_clf.predict(X_test)

### 7.2.2 oob 평가
- BaggingClassifier는 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기만큼인 m개 샘플 선택
- oob 샘플: 선택되지 않은 훈련 샘플의 나머지
- oob_score=True: 훈련이 끝난 후 자동으로 oob 평가 수행
- 결정 함수: 각 훈련 샘플의 클래스 확률

In [15]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    bootstrap=True,n_jobs=-1,oob_score=True)

In [16]:
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.888

In [18]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.904

In [19]:
bag_clf.oob_decision_function_

array([[0.01704545, 0.98295455],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.95939086, 0.04060914],
       [1.        , 0.        ],
       [0.19883041, 0.80116959],
       [0.        , 1.        ],
       [0.28915663, 0.71084337],
       [0.63934426, 0.36065574],
       [0.44059406, 0.55940594],
       [0.77419355, 0.22580645],
       [0.01162791, 0.98837209],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.80978261, 0.19021739],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.00540541, 0.99459459],
       [0.00537634, 0.99462366],
       [0.        , 1.        ],
       [0.99408284, 0.00591716],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.85483871, 0.14516129],
       [0.14754098, 0.85245902],
       [0.01775148, 0.98224852],
       [0.05913978, 0.94086022],
       [1.

## 7.3 랜덤 패치와 랜덤 서브스페이스
- 특성 샘플링 지원
- max_features, bootstrap_features 두 매개변수로 조절
- 특성에 대한 샘플링(각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련)
- 고차원 데이터셋 다룰 때 유용
- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링하는 것
- 랜덤 서브스페이스 방식: 훈련 샘플 모두 사용(bootstrap=False,max_samples=1.0)/
 특성은 샘플링(bootstrap_features=True, max_features<1)

 - 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산 낮춤

## 7.4 랜덤포레스트
- 배깅(또는 페이스팅) 방법을 적용한 결정 트리의 앙상블
- 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택해 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성 주입
 -> 트리 다양, 편향 손해 대신 분산 낮춰 더 훌륭한 모델

In [20]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf=rnd_clf.predict(X_test)

In [21]:
#랜덤포레스트와 유사
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features="auto",max_leaf_nodes=16),
    n_estimators=500,max_samples=1.0,bootstrap=True,n_jobs=-1)

### 7.4.1 엑스트라 트리
- 랜덤포레스트에서 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용
- 익스트림 랜덤 트리 앙상블: 트리를 더 무작위하게 만들기 위해 후보 특성을 사용해 무작위로 분할 후 최상의 분할 선택
  - 편향이 늘어나지만 분산 낮추게 됨
  - 모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요됨>랜덤 포레스트보다 훨씬 빠름

### 7.4.2 특성 중요도
- 특성의 상대적 중요도 측정 쉬움
- 사이킷런
  - 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도 측정(가중치 평균)
  - 훈련이 끝난 뒤 특성마다 자동으로 점수 계사, 중요도의 전체 합이 1이 되도록 결괏값 정규화 -->feature_importances 


In [22]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])
for name,score in zip(iris["feature_names"],rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.09194511308991098
sepal width (cm) 0.023513792972713058
petal length (cm) 0.4208869042321851
petal width (cm) 0.463654189705191


## 7.5 부스팅
- 약한 학습기 여러개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델 보완해나가며 일련의 예측기 학습시키는 것

### 7.5.1 에이다부스트
- 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것
- 경사하강법과 비슷
- 사이킷런: 클래스 확률에 기반

In [24]:
#max_depth=1: 결정 노드 하나와 리프 노드 두개로 이루어짐
from sklearn.ensemble import AdaBoostClassifier
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm="SAMME.R",learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅
- 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가
- 이전 예측기가 만든 잔여 오차에 새로운 예측기 학습


- learning_rate: 각 트리의 기여 정도 조절
  - 낮게 설정) 많은 트리 필요, 일반적으로 예측 성능 좋음 --- "축소"
- 최적의 트리 수를 찾기 위해 조기 종료 기법 사용; staged_predicted()


- 확률적 그레이디언트 부스팅: 훈련 샘플을 적게 설정하면, 편향 높 분산 낮/훈련 속도를 상당히 높


In [27]:
#첫번째 예측기
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [28]:
#두번째 예측기
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [30]:
#세번째 예측기
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3))

In [32]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1, max_depth=2, n_estimators=3)

In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

errors=[mean_squared_error(y_val,y_pred)
        for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=51)

In [37]:
#warm_start=True : 훈련 추가
gbrt=GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error=float("inf")
error_going_up=0
for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train,y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val,y_pred)
  if val_error < min_val_error:
    min_val_error=val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up==5:
      break

In [39]:
#최적화된 그레이디언트 부수팅 구현으로 XGBoost 파이썬 라이브러리 유명
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred=xgb_reg.predict(X_val)

[08:43:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [40]:
#자동 조기 종료
xgb_reg.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[08:45:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.470711
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.446117
[2]	validation_0-rmse:0.426248
[3]	validation_0-rmse:0.409439
[4]	validation_0-rmse:0.393658
[5]	validation_0-rmse:0.380216
[6]	validation_0-rmse:0.37125
[7]	validation_0-rmse:0.362521
[8]	validation_0-rmse:0.355159
[9]	validation_0-rmse:0.35038
[10]	validation_0-rmse:0.344469
[11]	validation_0-rmse:0.341888
[12]	validation_0-rmse:0.337277
[13]	validation_0-rmse:0.334131
[14]	validation_0-rmse:0.333638
[15]	validation_0-rmse:0.33167
[16]	validation_0-rmse:0.331038
[17]	validation_0-rmse:0.326692
[18]	validation_0-rmse:0.323648
[19]	validation_0-rmse:0.324173
[20]	validation_0-rmse:0.321179
[21]	validation_0-rmse:0.320841
[22]	validation_0-rmse:0.320236
[23]	validation_0-rmse:0.320037
[24]	validation_0-rmse:0.318084
[25]	validation_0-rmse

## 스태킹
- 앙상블에 속한 모든 예츢의 예측을 취합하는 모델을 훈련시킬 수 없을까요?라는 기본 아이디어에서 출발
- 블레더를 학습시키는 일반적인 방법은 홀드 아웃 세트 사용